# Import libraries

In [2]:
# Load Libraries
import sys
import os
sys.path.append(os.path.abspath("../"))

import numpy as np
import rasterio
from scipy.spatial import cKDTree

# Import preprocessing functions
from functions.preprocessing_functions import (
    load_raster_data,
    generate_and_export_mask,
    find_and_validate_seed,
    sea_core_from_seed,
    land_core_from_sea,
    extract_raster_coordinates,
    convert_wl_coordinates,
    save_aligned_stations_to_shapefile,
    water_perimeter_mask          # NEW
)

# Import configuration constants
from config import (
    DEM_FILE,
    LANDCOVER_FILE,
    SEED_CSV,
    SEA_CORE_TIF,
    LAND_CORE_TIF,
    ALIGNED_WATER_STATIONS_SHP,
    DEM_EPSG,
    WATER_CLASS_ID,
    WATER_BODY_PERIMETER_TIF # New
)

C:\Users\as229858\AppData\Local\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


## Preprocessing files for Coastal Flood Map (CFM)

In [4]:
# --------------------------------------------------
# Load DEM and Land Cover Data
# --------------------------------------------------
z_dem, transform, crs = load_raster_data(DEM_FILE)

with rasterio.open(LANDCOVER_FILE) as src:
    land_cover = src.read(1).astype(float)
    land_cover[land_cover == 255] = np.nan

# --------------------------------------------------
# Extract Raster Coordinates
# --------------------------------------------------
dem_lon, dem_lat, _ = extract_raster_coordinates(z_dem, transform, crs)

# --------------------------------------------------
# Identify and Validate Seed Ocean Point
# --------------------------------------------------
row_seed, col_seed = find_and_validate_seed(
    z_dem, land_cover, transform, crs,
    output_csv=SEED_CSV
)

# --------------------------------------------------
# Generate and Save Sea Core Mask
# --------------------------------------------------
sea_core_mask = sea_core_from_seed(z_dem, row_seed, col_seed)
generate_and_export_mask(
    name=SEA_CORE_TIF.replace(".tif", ""),
    compute_fn=lambda: sea_core_mask,
    reference_tif=DEM_FILE
)

# --------------------------------------------------
# Generate and Save Land Core Mask
# --------------------------------------------------
land_core_mask = land_core_from_sea(z_dem, sea_core_mask)
generate_and_export_mask(
    name=LAND_CORE_TIF.replace(".tif", ""),
    compute_fn=lambda: land_core_mask,
    reference_tif=DEM_FILE
)

# --------------------------------------------------
# NEW: Generate composite perimeter mask
# --------------------------------------------------
perimeter_mask = water_perimeter_mask(
    land_cover=land_cover,
    z_dem=z_dem,
    water_class=WATER_CLASS_ID,
    min_cluster_size=1,   # keep all
    connectivity=2         # 8-pixel rule
)

generate_and_export_mask(
    name=WATER_BODY_PERIMETER_TIF.replace(".tif", ""),
    compute_fn=lambda: perimeter_mask,
    reference_tif=DEM_FILE
)

# --------------------------------------------------
# Build KDTree from DEM Coordinates
# --------------------------------------------------
dem_shape = z_dem.shape
points = np.column_stack((dem_lon.ravel(), dem_lat.ravel()))
tree = cKDTree(points)

# --------------------------------------------------
# Define Water Level Observations
# --------------------------------------------------
wl_s2 = np.array([
    [-94.51333333, 29.51500000, 3.378],
    [-94.72500000, 29.35666667, 2.81],
    [-94.79333333, 29.31000000, 3.205],
    [-94.91666667, 29.48166667, 3.234],
    [-94.92020160, 29.44745403, 3.898],
    [-94.98500000, 29.68166667, 2.782],
    [-95.26500000, 29.72666667, 3.811],
    [-94.7149197, 29.86744193, 3.666],
    [-94.7743660, 29.9418847, 3.892],
    [-94.8182567, 30.05771539, 5.904],
    [-94.8507622, 30.4252067, 5.773],
    [-94.9857602, 29.97271914, 7.159752],
    [-94.9996503, 29.77078197, 5.5626],
    [-95.0938189, 29.87633426, 4.422648],
    [-95.1246139, 30.14523056, 5.321808],
    [-95.1682662, 30.2327137, 5.344058],
    [-95.1413198, 29.91633316, 14.71879],
    [-95.2339608, 29.93386089, 16.87373],
    [-95.2332671, 29.8371695, 11.04595],
    [-95.2163230, 29.69467363, 5.827776],
    [-95.2291011, 29.65661915, 7.434072],
    [-95.1785440, 29.51745517, 4.995672],
    [-95.2679907, 29.79328217, 12.11885],
    [-95.3068796, 29.9182784, 17.80337],
    [-95.3349365, 29.86189143, 18.90979],
    [-95.3132696, 29.80883745, 12.89609],
    [-95.2910473, 29.74939477, 4.498848],
    [-95.2893807, 29.67439687, 7.046976],
    [-95.2974366, 29.5968991, 13.01801],
    [-95.3680556, 29.79277778, 13.09421],
    [-95.3585490, 29.76661676, 9.232392],
    [-95.3388889, 29.71416667, 9.162288],
    [-95.3230476, 29.37154349, 7.382866],
    [-95.4288270, 30.0357753, 8.59536],
    [-95.4179936, 29.95688886, 25.44775],
    [-95.3971612, 29.77522777, 14.0208],
    [-95.4085505, 29.76022829, 9.878568],
    [-95.4121620, 29.69717469, 12.5029],
    [-95.4460522, 29.61884399, 11.16787],
    [-95.5235538, 29.7468959, 16.67256],
    [-95.5282765, 29.6727317, 17.24254],
    [-95.5577213, 29.76217336, 18.70558],
    [-95.5830000, 29.7091197, 18.92503],
    [-95.5621664, 29.65662136, 21.717],
    [-95.6057782, 29.7618958, 20.59229],
    [-95.6466120, 29.86717035, 7.583424],
    [-95.6257783, 29.8357824, 31.13227],
    [-95.6868912, 29.8307828, 4.440936],
    [-94.19281384, 29.68444965,  5.096256 ],
    [-94.39030462, 29.59445505,  4.90728  ],
    [-94.68690134, 29.77280093,  4.764024 ],
    [-94.67528817, 29.60418343,  3.895344 ],
    [-94.64808849, 29.46585341,  4.105656 ],
    [-94.63418624, 29.45137871,  5.824728 ],
    [-94.75107958, 29.3344534,   4.696968 ],
    [-94.99329828, 29.71304287,  3.797808 ],
    [-94.99889929, 29.62030003,  3.925824 ],
    [-95.02471845, 29.55167198,  3.776472 ],
    [-94.95779403, 29.50637545,  3.401568 ],
    [-94.90529215, 29.3039072,   3.834384 ],
    [-94.87779852, 29.23803827,  4.416552 ],
    [-95.12832542, 29.59193594,  4.166616 ],
    [-95.103909,   29.51331277,  3.21564  ],
    [-95.04778292, 29.45666901,  3.21564  ],
    [-95.04001319, 29.35584376,  3.176016 ],
    [-94.94470485, 29.22085215,  3.770376 ],
    [-95.13140589, 29.28667628,  2.25552  ],
    [-95.35660191, 29.29667801,  8.071104 ],
    [-95.20831237, 29.21195597,  2.2098   ],
    [-95.11719144, 29.08613596,  2.883408 ],
    [-95.28420906, 29.33635959,  2.828544 ]
])

# --------------------------------------------------
# Convert Water Level Coordinates to DEM CRS
# --------------------------------------------------
wl_x, wl_y, wl_values = convert_wl_coordinates(wl_s2, crs)

# --------------------------------------------------
# Align Water Level Points to DEM Grid
# --------------------------------------------------
aligned_stations = []
for orig_x, orig_y, wl_value in zip(wl_x, wl_y, wl_values):
    distance, idx = tree.query([orig_x, orig_y])
    row, col = np.unravel_index(idx, dem_shape)
    snapped_x = dem_lon[row, col]
    snapped_y = dem_lat[row, col]
    aux_value = z_dem[row, col]
    aligned_stations.append([snapped_x, snapped_y, wl_value, aux_value])

aligned_stations = np.array(aligned_stations)

# --------------------------------------------------
# Save Aligned Water Stations to Shapefile
# --------------------------------------------------
save_aligned_stations_to_shapefile(
    aligned_stations=aligned_stations,
    output_shapefile=ALIGNED_WATER_STATIONS_SHP,
    crs_epsg=DEM_EPSG
)

Coordinates extracted in UTM
Seed point verified: Water class (ID=21). Elevation = -18.47 m
Seed point saved to ../output_data\seed_ocean.csv
../output_data\01_dem_sea_core.tif generated and saved | total time: 1.74 seconds
../output_data\01_dem_land_core.tif generated and saved | total time: 1.77 seconds
../output_data\02_water_body_perimeter.tif generated and saved | total time: 1.86 seconds
Transformed 71 water level coordinates to raster CRS.
Saved aligned water stations to ../output_data\Aligned_Water_Stations.shp


C:\Users\as229858\Documents\PyFlood\functions\preprocessing_functions.py:284: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(output_shapefile)


## Preprocessing files for Coastal Flood Map + Attenuation (CFM+A)

In [6]:
# Load Postcalibration Functions
from functions.preprocessing_attenuation_functions import (
    load_seed_point,
    perimeter_raster_to_xyz,
    save_water_body_perimeter_as_shapefile,
    calc_distance_to_water_body_parallel,
    save_distance_raster,
    align_hwm_to_dem
)

# Import configuration constants
from config import (
    SEED_CSV,
    DEM_FILE,
    WATER_BODY_SHP,
    DISTANCE_TO_WATER_BODY,
    HWM_INPUT_FILE,
    ALIGNED_HWM_SHP,
    DEM_EPSG,
    WATER_BODY_PERIMETER_TIF
)

# --------------------------------------------------
# Load Seed Ocean Point from CSV
# --------------------------------------------------
seed_ocean, seed_elevation = load_seed_point(SEED_CSV)

water_body_perimeter_xyz, water_body_perimeter_pixel_indices = perimeter_raster_to_xyz(
    perim_tif=WATER_BODY_PERIMETER_TIF,
    dem_lon=dem_lon,
    dem_lat=dem_lat,
    z_dem=z_dem,
    nodata_val=0
)

print(f"Loaded {len(water_body_perimeter_xyz):,} Water body perimeter points.")


# --------------------------------------------------
# Save Water Body as Shapefile
# --------------------------------------------------
save_water_body_perimeter_as_shapefile(
    water_body_perimeter_xyz,
    output_shapefile=WATER_BODY_SHP,
    epsg_code=DEM_EPSG
)

# --------------------------------------------------
# Compute Distance to Water Body
# --------------------------------------------------
distances = calc_distance_to_water_body_parallel(
    dem_lon, dem_lat, z_dem, water_body_perimeter_xyz[:, :2]
)

# Reshape and Save Distance Raster
distances = distances.reshape(z_dem.shape)

print(f"Minimum distance to water body: {np.nanmin(distances):.2f} meters")
print(f"Maximum distance to water body: {np.nanmax(distances):.2f} meters")

save_distance_raster(
    distances=distances,
    dem_file=DEM_FILE,
    output_filename=DISTANCE_TO_WATER_BODY
)

gdf_aligned, hwm_indices = align_hwm_to_dem(
    shapefile_path=HWM_INPUT_FILE,
    dem_lon=dem_lon,
    dem_lat=dem_lat,
    z_dem=z_dem,
    crs=crs,
    output_shapefile=ALIGNED_HWM_SHP
)

Loaded seed point: (328373.13, 3247140.13), Elevation: -18.47 m
Loaded 2,917,451 Water body perimeter points.


C:\Users\as229858\Documents\PyFlood\functions\preprocessing_attenuation_functions.py:104: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(output_shapefile)


Water body perimeter shapefile saved as ../output_data\water_body.shp
Processing 449,544,536 valid land points in parallel...
Distance computation completed in 87.04 seconds.
Minimum distance to water body: 0.00 meters
Maximum distance to water body: 5466.16 meters
Distance raster saved at: ../output_data\Distance_to_Water_Body.tif
HWM aligned shapefile saved to: ../output_data\Aligned_HWM_with_indices.shp
